In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, classification_report, r2_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Load datasets
df_train = pd.read_csv('stock_roi_risk_training_data.csv')
df_test = pd.read_csv('stock_roi_risk_test_data.csv')

# Compute ROI and profit
for df in [df_train, df_test]:
    df['roi'] = df['percentage_change'] * 100
    df['profit_dollars'] = df['invested_amount'] * df['percentage_change']
    df['roi'] = df['roi'].fillna(0)
    df['roi'] = df['roi'].apply(lambda x: x if x > -1 else -0.999)  # avoid log1p domain error
    df['log_roi'] = np.log1p(df['roi'])

# ✅ Subtract 1 from risk labels
df_train['risk'] = df_train['risk'] - 1
df_test['risk'] = df_test['risk'] - 1

# Features
features = ['stock', 'days', 'invested_amount']
X_train = df_train[features].values
X_test = df_test[features].values

# Standardize input
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.pkl')

# ROI targets
y_roi_train = df_train['log_roi'].values
y_roi_test = df_test['roi'].values
log_y_roi_test = df_test['log_roi'].values

# Risk targets (adjusted to 5 classes)
y_risk_train_raw = df_train['risk'].values
y_risk_train = to_categorical(y_risk_train_raw, num_classes=5)

y_risk_test_raw = df_test['risk'].values
y_risk_test = to_categorical(y_risk_test_raw, num_classes=5)

# Class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_risk_train_raw), y=y_risk_train_raw)
class_weights_dict = dict(enumerate(class_weights))
roi_model = Sequential([
    Dense(128, activation='relu', input_shape=(3,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)
])

roi_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

early_stop_roi = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

roi_model.fit(
    X_train_scaled,
    y_roi_train,
    validation_split=0.1,
    epochs=100,
    batch_size=1024,
    callbacks=[early_stop_roi],
    verbose=1
)

roi_model.save('roi_model.keras')
risk_model = Sequential([
    Dense(128, activation='relu', input_shape=(3,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')  # Updated: 5 classes
])

risk_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stop_risk = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

risk_model.fit(
    X_train_scaled,
    y_risk_train,
    validation_split=0.1,
    epochs=100,
    batch_size=1024,
    callbacks=[early_stop_risk],
    class_weight=class_weights_dict,
    verbose=1
)

risk_model.save('risk_model.keras')
